In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

# Define data generators with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training and test datasets
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=64,  # Adjusted batch size
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    'dataset/valid',
    target_size=(64, 64),
    batch_size=64,  # Adjusted batch size
    class_mode='categorical'
)

# Define a more complex CNN model architecture
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=len(training_set.class_indices), activation='softmax')
])

# Reduce the learning rate
legacy_adam_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

# Compile the model
cnn.compile(optimizer=legacy_adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = cnn.fit(x=training_set, validation_data=test_set, epochs=100)

# Evaluate the model on test set using additional metrics
loss, accuracy = cnn.evaluate(test_set, steps=len(test_set))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Extracting class labels
class_labels = training_set.class_indices

# Verify the number of classes inferred from the model
num_classes = len(class_labels)
print("Number of classes inferred from the model:", num_classes)

# Extract true labels and predicted labels
y_true = test_set.classes
print("1", y_true)
y_pred = cnn.predict(test_set)
print("2", y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
print("3", y_pred_classes)

# Generate classification report
print(classification_report(y_true, y_pred_classes, target_names=training_set.class_indices.keys()))


Found 70295 images belonging to 38 classes.
Found 17571 images belonging to 38 classes.
Epoch 1/100
1099/1099 [==============================] - 123s 111ms/step - loss: 2.7760 - accuracy: 0.2199 - val_loss: 2.2050 - val_accuracy: 0.3645
Epoch 2/100
1099/1099 [==============================] - 120s 109ms/step - loss: 1.9376 - accuracy: 0.4312 - val_loss: 2.0627 - val_accuracy: 0.4198
Epoch 3/100
1099/1099 [==============================] - 132s 120ms/step - loss: 1.6087 - accuracy: 0.5191 - val_loss: 2.1592 - val_accuracy: 0.4214
Epoch 4/100
1099/1099 [==============================] - 140s 127ms/step - loss: 1.4033 - accuracy: 0.5734 - val_loss: 1.6993 - val_accuracy: 0.5268
Epoch 5/100
1099/1099 [==============================] - 134s 122ms/step - loss: 1.2536 - accuracy: 0.6193 - val_loss: 1.5891 - val_accuracy: 0.5559
Epoch 6/100
1099/1099 [==============================] - 132s 120ms/step - loss: 1.1321 - accuracy: 0.6502 - val_loss: 1.5491 - val_accuracy: 0.5625
Epoch 7/100
1099/1

In [2]:
tf.keras.models.save_model(cnn, "my_model.keras")


NameError: name 'tf' is not defined

In [33]:
import json

with open('class_indices.json', 'w') as f:
    json.dump(training_set.class_indices, f) 

In [4]:
import tensorflow as tf

# Load the model
loaded_model = tf.keras.models.load_model("my_model.keras")


In [3]:
import numpy as np
from keras.preprocessing import image

# Load the single prediction image
test_image_path = 'dataset/single_prediction/CornCommonRust3.JPG'
test_image = image.load_img(test_image_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Make prediction
result = cnn.predict(test_image)
predicted_class_index = np.argmax(result)

# Retrieve class labels
class_labels = training_set.class_indices

# Get predicted class label
predicted_class_label = list(class_labels.keys())[predicted_class_index]

print("Predicted disease:", predicted_class_label)

NameError: name 'cnn' is not defined